In [20]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
from skyplane.compute.gcp.gcp_auth import GCPAuthentication

# Initialize necessary things
auth = GCPAuthentication()
default_instance = auth.config.get_flag("gcp_instance_class")
default_region = auth.config.get_flag("gcp_default_region")

# Default instances vcpus count
instance = auth.get_gcp_client().machineTypes().get(project=auth.project_id, zone=default_region, machineType=default_instance).execute()
print(instance)
# vcpus = instance.get("guestCpus", None)
# if vcpus is not None:
#     print(f"Instance of type {default_instance} has {vcpus} vcpus")

# # Retrieve information about the specified region
# instances = auth.get_gcp_instances(gcp_region=default_region)
# items = instances.get("items", None)

# # Get the number of vCPUs of the instance you created
# if items is not None:
#     for item in items:
#         machine_type = item.get("machineType", None)
#         if machine_type is not None:
#             machine_type = item["machineType"].split("/")[-1]
#             instance = auth.get_gcp_client().machineTypes().get(project=auth.project_id, zone=default_region, machineType=machine_type).execute()
#             vcpus = instance.get("guestCpus", None)
#             if vcpus is not None:
#                 print(f"Instance of type {machine_type} has {vcpus} vcpus")

In [ ]:
import json
from skyplane.config_paths import gcp_instances_path

# Hard coding the vcpu info since the api doesn"t provide a clean way for this
vcpus_info = {
    "n1-standard-1": 1,
    "n1-standard-2": 2,
    "n1-standard-4": 4,
    "n1-standard-8": 8,
    "n1-standard-16": 16,
    "n1-standard-32": 32,
    "n1-standard-64": 64,
    "n1-standard-96": 96,
    "n1-highcpu-2": 2,
    "n1-highcpu-4": 4,
    "n1-highcpu-8": 8,
    "n1-highcpu-16": 16,
    "n1-highcpu-32": 32,
    "n1-highcpu-64": 64,
    "n1-highcpu-96": 96,
    "n1-highmem-2": 2,
    "n1-highmem-4": 4,
    "n1-highmem-8": 8,
    "n1-highmem-16": 16,
    "n1-highmem-32": 32,
    "n1-highmem-64": 64,
    "n1-highmem-96": 96
}
with gcp_instances_path.open("w") as f:
    f.write(json.dumps(vcpus_info))

In [ ]:
from aws.aws_auth import AWSAuthentication

# Initialize necessary things
auth = AWSAuthentication()
default_instance = auth.config.get_flag("aws_instance_class")
default_region = auth.config.get_flag("aws_default_region")

# Default instances vcpus count
ec2 = auth.get_boto3_client("ec2", default_region)
response = ec2.describe_instance_types(InstanceTypes=[default_instance])
vcpu_count = response["InstanceTypes"][0]["VCpuInfo"]["DefaultVCpus"]
print(f"The {default_instance} instance has {vcpu_count} vCPUs.")

In [ ]:
import json
from skyplane.config_paths import aws_instances_path

with aws_instances_path.open("r") as f:
    vcpus_info = json.load(f)

    types = {}

    for instance, vcpus in vcpus_info.items():
        header = instance.split(".")[0][:2]
        types[header] = types.get(header, [])
        types[header].append((instance, vcpus))

    print(json.dumps(types, sort_keys=True, indent=2))

In [ ]:
import json
from skyplane.compute.aws.aws_auth import AWSAuthentication
from skyplane.config_paths import aws_instances_path
import boto3

# Initialize necessary things
auth = AWSAuthentication()
default_instance = auth.config.get_flag("aws_instance_class")
default_region = auth.config.get_flag("aws_default_region")

# Use the describe_instance_types() method to get information about instance types
ec2 = boto3.client("ec2", region_name="us-east-1")
response = ec2.describe_instance_types()

# Extract the vCPU information from the response
vcpus = {}
for instance_type in response["InstanceTypes"]:
    instance_type_name = instance_type["InstanceType"]
    vcpus[instance_type_name] = instance_type["VCpuInfo"]["DefaultVCpus"]

print(len(vcpus.keys()))

# Print the vCPU information
for instance_type, vcpu_count in vcpus.items():
    print(f"{instance_type}: {vcpu_count} vCPUs")

with aws_instances_path.open("w") as f:
    f.write(json.dumps(vcpus, indent=2))

In [ ]:
instance_type = default_instance
with aws_instances_path.open("r") as f:
    vcpus_info = json.load(f)
    print(vcpus_info[instance_type])

In [ ]:
import boto3

# Create a client object for the EC2 service
ec2 = boto3.client("ec2")

# Use the describe_instances() method to get information about instances
response = ec2.describe_instances()

# Extract the instance IDs from the response
instance_ids = []
for reservation in response["Reservations"]:
    for instance in reservation["Instances"]:
        instance_ids.append(instance["InstanceId"])

# Print the instance IDs
print("Instances in this account:")
for instance_id in instance_ids:
    print(instance_id)


In [ ]:
from google.cloud import compute_v1
from skyplane.compute.gcp.gcp_auth import GCPAuthentication

# Initialize necessary things
auth = GCPAuthentication()
default_instance = auth.config.get_flag("gcp_instance_class")
default_region = auth.config.get_flag("gcp_default_region")
auth.get_gcp_instances(gcp_region=default_region)["items"][0]

# Use the list() method to get information about instances in the specified region
# instances = []
# for zone_name in zone_names:
#     instances_in_zone = client.list(project="skyplane-test", zone=zone_name)
#     instances.extend(instances_in_zone)

# # Print the instance names
# print("Instances in the specified region:")
# for instance in instances:
#     if instance.name:
#         print(instance.name)


In [ ]:
import boto3

client = boto3.client("ec2", region_name="us-east-1")

response = client.describe_instance_types(
    MaxResults=100
)

instance_types = response["InstanceTypes"]
while "NextToken" in response:
    response = client.describe_instance_types(
        MaxResults=100,
        NextToken=response["NextToken"]
    )
    instance_types.extend(response["InstanceTypes"])

vcpus = {}
for instance_type in instance_types:
    instance_type_name = instance_type["InstanceType"]
    vcpus[instance_type_name] = {
        "vcpus": instance_type["VCpuInfo"]["DefaultVCpus"],
        "usage_classes": instance_type["SupportedUsageClasses"]
    }

print(json.dumps(vcpus, indent=2))

In [ ]:
import boto3
import json

client = boto3.client("ec2", region_name="us-east-1")

response = client.describe_instance_types(
    MaxResults=100
)

instance_types = response["InstanceTypes"]
while "NextToken" in response:
    response = client.describe_instance_types(
        MaxResults=100,
        NextToken=response["NextToken"]
    )
    instance_types.extend(response["InstanceTypes"])

families = {}
for instance_type in instance_types:
    instance_type_name = instance_type["InstanceType"]
    family = instance_type_name.split(".")[0]
    if family not in families:
        families[family] = {}
    families[family][instance_type_name] = instance_type["VCpuInfo"]["DefaultVCpus"]

print(json.dumps(families, indent=2))


In [ ]:
region_list = ["australiacentral",
"australiacentral2",
"australiaeast",
"australiasoutheast",
"brazilsouth",
"brazilsoutheast",
"brazilus",
"canadacentral",
"canadaeast",
"centralindia",
"centralus",
"eastasia",
"eastus",
"eastus2",
"francecentral",
"francesouth",
"germanynorth",
"germanywestcentral",
"japaneast",
"japanwest",
"jioindiacentral",
"jioindiawest",
"koreacentral",
"koreasouth",
"northcentralus",
"northeurope",
"norwayeast",
"norwaywest",
"polandcentral",
"qatarcentral",
"southafricanorth",
"southafricawest",
"southcentralus",
"southeastasia",
"southindia",
"swedencentral",
"switzerlandnorth",
"switzerlandwest",
"uaecentral",
"uaenorth",
"uksouth",
"ukwest",
"westcentralus",
"westeurope",
"westindia",
"westus",
"westus2",
"westus3"]

In [ ]:
# Azure initialization
from skyplane.config_paths import aws_quota_path, gcp_quota_path, azure_quota_path
import json

result = []
with open(azure_quota_path, "r") as f:
    result = json.load(f)

print(result)

# all_region_vcpus_azure = {}
# for region in region_list:
#     azure_target = [item for item in result[region] if item["properties"]["name"]["value"] == "standardDv5Family"]
#     if azure_target:
#         all_region_vcpus_azure[region] = azure_target[0]["properties"]["limit"]["value"]
# all_region_vcpus_azure

In [ ]:
# AWS get quota
region = "us-east-1"
spot = False
with aws_quota_path.open("r") as f:
    quota_limits = json.load(f)
    print(quota_limits)
    for quota in quota_limits:
        if quota["region_name"] == region:
            if spot:
                print(quota["spot_standard_vcpus"])
            else:
                print(quota["on_demand_standard_vcpus"])

In [ ]:
# GCP get quota 
region = "us-east1-a"
spot = True
with gcp_quota_path.open("r") as f:
    quota_limits = json.load(f)
    print(quota_limits)
    region_family = "-".join(region.split("-")[:2])
    if region_family in quota_limits:
        print(quota_limits[region_family])

In [ ]:
# Azure get quota
from skyplane.config_paths import azure_standardDv5_quota_path, azure_config_path
region = "uaenorth"
spot = False
with open(azure_standardDv5_quota_path, "r") as f:
    quota_limits = json.load(f)
    if region in quota_limits:
        print(quota_limits[region])

In [ ]:
# AWS fallback
from skyplane.compute import AWSAuthentication

quota_limit = 2
instance_type = "m5.8xlarge"
if AWSAuthentication._AWS_VCPUS[instance_type] <= quota_limit:
    print(None)  # don"t need to fall back

max_vcpus, max_instance = 0, None
for instance, vcpus in AWSAuthentication._AWS_VCPUS.items():
    # Get the largest of the smaller VMs than the quota_limit
    if vcpus <= quota_limit and vcpus > max_vcpus:
        max_vcpus = vcpus
        max_instance = instance
print(max_instance, max_vcpus)  # None if no smaller VM exists

In [ ]:
# GCP fallback

quota_limit = 24
instance_type = "n2-standard-32"
# By default, using n2-standard series; hence, can just hard code the pattern
vcpus = int(instance_type.split("-")[-1])
if vcpus <= quota_limit:
    print(None)  # don"t need to fall back

# Find the greatest instance that is less than quota
for val in (128, 96, 80, 64, 48, 32, 16, 8, 4, 2):
    if val <= quota_limit:
        print(f"n2-standard-{val}")

In [ ]:
# Azure fallback
import re


quota_limit = 24
instance_type = "Standard_D32_v5"
# Since Azure instances follow a common pattern, we can just extract the vCPUs from the instance name
vcpus = int(re.search(r"\d+", instance_type).group())
if vcpus <= quota_limit:
    print(None)  # don"t need to fall back

# Find the greatest instance that is less than quota
for val in (96, 64, 48, 32, 16, 8, 4, 2):
    if val <= quota_limit:
        print(f"Standard_D{val}_v5")

In [ ]:
from skyplane.compute.gcp.gcp_auth import GCPAuthentication

# Initialize necessary things
auth = GCPAuthentication()
default_instance = auth.config.get_flag("gcp_instance_class")
default_region = auth.config.get_flag("gcp_default_region")

service = auth.get_gcp_client("compute", "v1")
request = service.regions().list(project=auth.project_id)
region_to_vcpus = {}
while request is not None:
    response = request.execute()
    for region in response["items"]:
        if region["kind"] != "compute#region":
            continue
        region_name = region["name"]
        for quota_item in region["quotas"]:
            if quota_item["metric"] == "CPUS":
                region_to_vcpus[region_name] = quota_item["limit"]
                break
    request = service.regions().list_next(previous_request=request, previous_response=response)

In [ ]:
region_to_vcpus

In [ ]:
import boto3

ec2_client = boto3.client("ec2")
account_attributes = ec2_client.describe_account_attributes(AttributeNames=["max-instances"])

for attribute in account_attributes["AccountAttributes"]:
    if attribute["AttributeName"] == "max-instances":
        for value in attribute["AttributeValues"]:
            if value["AttributeValue"] == "vCPUs":
                vcpu_limit = int(value["Max"])
                print(f"vCPU limit: {vcpu_limit}")


In [ ]:
from skyplane import compute

def test_fall_back_aws():

    # Different instances
    instances = (
        "Standard_D2_v5",
        "Standard_D4_v5",
        "Standard_D8_v5",
        "Standard_D16_v5",
        "Standard_D32_v5",
        "Standard_D48_v5",
        "Standard_D64_v5",
        "Standard_D96_v5",
    )

    # Default parameters
    auth = compute.AzureAuthentication
    quota_limit = 4

    # Test fall back
    for vm_type in instances:
        smaller_vm = auth.fall_back_to_smaller_vm_if_neccessary(instance_type=vm_type, quota_limit=quota_limit)

        if vm_type == "Standard_D4_v5" or vm_type == "Standard_D2_v5":
            assert smaller_vm is None
        else:
            assert smaller_vm is not None
            assert smaller_vm == "Standard_D4_v5"

In [ ]:
from skyplane import compute

def test_fall_back_aws():

    # Different instances
    instances = (
        "n2-standard-128",
        "n2-standard-96",
        "n2-standard-80",
        "n2-standard-64",
        "n2-standard-48",
        "n2-standard-32",
        "n2-standard-16",
        "n2-standard-8",
        "n2-standard-4",
        "n2-standard-2"
    )

    # Default parameters
    auth = compute.GCPAuthentication
    quota_limit = 8

    # Test fall back
    for vm_type in instances:
        smaller_vm = auth.fall_back_to_smaller_vm_if_neccessary(instance_type=vm_type, quota_limit=quota_limit)

        if vm_type == "n2-standard-8" or vm_type == "n2-standard-4" or "n2-standard-2":
            assert smaller_vm is None
        else:
            assert smaller_vm is not None
            assert smaller_vm == "n2-standard-8"


In [ ]:
from skyplane import compute

def test_fall_back_aws():

    # Different instances
    instances = (
        "m5.8xlarge",
        "m5.12xlarge",
        "m5.24xlarge",
        "m5.metal",
        "m5.xlarge",
        "m5.4xlarge",
        "m5.large",
        "m5.2xlarge",
        "m5.16xlarge",
    )

    # Default parameters
    auth = compute.AWSAuthentication
    quota_limit = 5

    # Test fall back
    for vm_type in instances:
        smaller_vm = auth.fall_back_to_smaller_vm_if_neccessary(instance_type=vm_type, quota_limit=quota_limit)

        if vm_type == "m5.xlarge" or vm_type == "m5.large":
            assert smaller_vm is None
        else:
            assert smaller_vm is not None
            assert smaller_vm == "m5.xlarge"


In [ ]:
def calculate_vcpu_portions(num_vcpus, quota_limit, vcpu_options):
    # Find the largest vCPU option
    max_vcpus = max(vcpu_options)
    
    # If the desired vCPU count is within the quota limit, try to use a single vCPU option
    if num_vcpus <= quota_limit:
        # If the desired vCPU count matches one of the available options, use that option
        # if num_vcpus in vcpu_options:
        return (1, num_vcpus) # [num_vcpus]
        # # Otherwise, try to use the largest available vCPU option that is smaller than the desired count
        # for vcpu_count in sorted(vcpu_options, reverse=True):
        #     if vcpu_count <= num_vcpus:
        #         return (num_vcpus // vcpu_count, vcpu_count) # [vcpu_count] * (num_vcpus // vcpu_count)
    
    # # If the largest available vCPU option is within the quota limit, use it to launch as many vCPUs as possible
    # elif max_vcpus <= quota_limit:
    #     return (num_vcpus // max_vcpus, max_vcpus) # [max_vcpus] * (num_vcpus // max_vcpus)
    
    # Otherwise, try to split the desired vCPU count into smaller portions that are within the quota limit and use the largest option
    else:
        for vcpu_count in sorted(vcpu_options, reverse=True):
            if vcpu_count <= quota_limit: # and vcpu_count <= num_vcpus:
                portions = num_vcpus // vcpu_count
                remaining_vcpus = num_vcpus - (vcpu_count * portions)
                # If the remaining vCPUs are 0, use the current option to launch all portions
                if remaining_vcpus == 0:
                    return (portions, vcpu_count) # [vcpu_count] * portions
    # Return an empty list if no valid vCPU portions were found
    return ()


In [ ]:
# Example usage
hey = (128, 96, 80, 64, 48, 32, 16, 8, 4, 2)

for num_vcpus in hey:
    for quota_limit in range(2, num_vcpus):
        num, vm_size = calculate_vcpu_portions(num_vcpus, quota_limit, hey)
        if num * vm_size != num_vcpus:
            print(f"{num_vcpus} with limit {quota_limit}: {num, vm_size}")

# num_vcpus = 48
# quota_limit = 35
# vcpu_portions = calculate_vcpu_portions(num_vcpus, quota_limit, (96, 64, 48, 32, 16, 8, 4, 2))
# print(vcpu_portions)
# Output: [16, 16]

In [ ]:
# Deprecated code
"""
- planner.py
if vcpus_used <= quota_limit:
    return None  # No need to fall back

for val in all_vcpus:
    if val <= quota_limit:
        return vm_family.format(val if cloud_provider != "aws" else f"{val // 4}x" if val // 4 else "2")

- dataplane.py:     
# create VMs from the topology
auths = {
    "aws": compute.AWSAuthentication,
    "azure": compute.AzureAuthentication,
    "gcp": compute.GCPAuthentication,
    "ibmcloud": compute.IBMCloudAuthentication,
}

spot = getattr(self.transfer_config, f"{cloud_provider}_use_spot_instances")
vm_type = getattr(self.transfer_config, f"{cloud_provider}_instance_class")

# Fall back to smaller vm if necessary
auth = auths[cloud_provider]
quota_limit = auth.get_quota_limits_for(region, spot)
if quota_limit is not None:
    smaller_vm = auth.fall_back_to_smaller_vm_if_neccessary(instance_type=vm_type, quota_limit=quota_limit)
    if smaller_vm is not None:
        vm_type = smaller_vm
        logger.warning(f"Falling back to {smaller_vm} at node {region} due to the vCPU quota limit {quota_limit}")

# TODO: Add the logic for partitioning the task into multiple vms if we fell back (in fall_back functions)
# Ex: if the config vm uses 32 vCPUs but the quota limit is 8 vCPUS, call add_task 4 times with the smaller vm

- planner.py
    # @staticmethod
    # def _get_vcpus_options(cloud_provider: str) -> Tuple[int]:
    #     return (
    #         Planner._AWS_VCPUS,
    #         "m5.{}large" if cloud_provider == "aws" else Planner._AZURE_VCPUS if cloud_provider == "azure" else Planner._GCP_VCPUS,
    #     )

    # _AWS_VCPUS: Tuple[int] = (96, 64, 48, 32, 16, 8, 4, 2)
    # _AZURE_VCPUS: Tuple[int] = (96, 64, 48, 32, 16, 8, 4, 2)
    # _GCP_VCPUS: Tuple[int] = (128, 96, 80, 64, 48, 32, 16, 8, 4, 2)

    
        # # Get the n_portions, n_vcpus pair for source and destination
        # src_n_portion, dst_n_vcpus = Planner._split_vcpus(num_vcpus=target_vcpus, quota_limit=quota_limit, vcpu_options=Planner._get_vcpus_options(src_provider))
        # dst_n_portion, dst_n_vcpus = Planner._split_vcpus(num_vcpus=target_vcpus, quota_limit=quota_limit, vcpu_options=Planner._get_vcpus_options(src_provider))
        # # Planner._split_vcpus(num_vcpus=target_vcpus, quota_limit=quota_limit

        # if quota_limit is not None:
        #     vm_portions = Planner._fall_back_to_smaller_vm_if_neccessary(
        #         cloud_provider=cloud_provider,
        #         instance_type=vm_type,
        #         quota_limit=quota_limit,
        #     )
        #     if vm_portions is not None:
        #         n_portions, vm_type = vm_portions
        #         if n_portions > 1:
        #             logger.warning(f"Falling back to {vm_type} at {region} due to the vCPU quota limit {quota_limit}")
        # return n_portions, vm_type

    # def _fall_back_to_smaller_vm_if_neccessary(
    #     self, cloud_provider: str, instance_type: str, quota_limit: int, is_destination: bool = False
    # ) -> Optional[str]:
    #     n_vcpus, all_vcpus, vm_family = (
    #         (re.search(r"\d+x", instance_type), Planner._AWS_VCPUS, "m5.{}large")
    #         if cloud_provider == "aws"
    #         else (re.search(r"\d+", instance_type), Planner._AZURE_VCPUS, "Standard_D{}_v5")
    #         if cloud_provider == "azure"
    #         else (int(instance_type.split("-")[-1]), Planner._GCP_VCPUS, "n2-standard-{}")
    #     )

    #     vm_portions = Planner._split_vcpus(n_vcpus if cloud_provider == "gcp" else int(n_vcpus.group()), quota_limit, all_vcpus)
    #     if vm_portions is ():
    #         return None

    #     num_portions, vm_size = vm_portions
    #     return (num_portions, vm_family.format(vm_size if cloud_provider != "aws" else f"{vm_size // 4}x" if vm_size // 4 else "2"))
"""


In [24]:
from skyplane import compute
from skyplane.api.config import TransferConfig
from skyplane.planner.planner import MulticastDirectPlanner, Planner
from skyplane.utils.fn import do_parallel

quota_limits = {
    "aws": [
        {"on_demand_standard_vcpus": 5, "spot_standard_vcpus": 5, "region_name": "us-east-1"},
    ],
    "gcp": {
        "us-east1": 8,
    },
    "azure": {
        "uaenorth": 4,
    },
}

transfer_config = TransferConfig()
planner = MulticastDirectPlanner(n_instances=8, n_connections=100, transfer_config=transfer_config)
planner.quota_limits = quota_limits

region_tags = ["aws:us-east-1", "azure:uaenorth", "gcp:us-east1"]
tests = {
    ("aws:us-east-1", "aws:us-east-1"): 0,
    ("aws:us-east-1", "azure:uaenorth"): 0,
    ("aws:us-east-1", "gcp:us-east1"): 0,
    ("azure:uaenorth", "aws:us-east-1"): 0,
    ("azure:uaenorth", "azure:uaenorth"): 0,
    ("azure:uaenorth", "gcp:us-east1"): 0,
    ("gcp:us-east1", "aws:us-east-1"): 0,
    ("gcp:us-east1", "azure:uaenorth"): 0,
    ("gcp:us-east1", "gcp:us-east1"): 0,
}

results = {}
for i, src_region_tag in enumerate(region_tags):
    dst_region_tags = region_tags[:i] + region_tags[i+1:]
    vm_info = do_parallel(planner._calculate_vm_types, [src_region_tag] + dst_region_tags)  # type: ignore
    vm_types = {v[0]: Planner._vcpus_to_vm(cloud_provider=v[0].split(":")[0], vcpus=v[1][0]) for v in vm_info}  # type: ignore
    n_instances = min(v[1][1] for v in vm_info)  # type: ignore

    print(vm_types, n_instances)


------('aws:us-east-1', 'aws:us-east-1')------


[(32, 5)]
32 5
8 4
m5.xlarge

------('aws:us-east-1', 'azure:uaenorth')------


[(2, 4)]
2 4
1 2
m5.large

------('aws:us-east-1', 'gcp:us-east1')------


[(16, 5)]
16 5
4 4
m5.xlarge

------('azure:uaenorth', 'aws:us-east-1')------


[(2, 4)]
2 4
1 2
Standard_D2_v5

------('azure:uaenorth', 'azure:uaenorth')------


[(2, 4)]
2 4
1 2
Standard_D2_v5

------('azure:uaenorth', 'gcp:us-east1')------


[(2, 4)]
2 4
1 2
Standard_D2_v5

------('gcp:us-east1', 'aws:us-east-1')------


[(16, 5)]
16 5
4 4
n2-standard-4

------('gcp:us-east1', 'azure:uaenorth')------


[(2, 4)]
2 4
1 2
n2-standard-2

------('gcp:us-east1', 'gcp:us-east1')------


[(16, 8)]
16 8
2 8
n2-standard-8


In [25]:
results

{('aws:us-east-1', 'aws:us-east-1'): (8, 4, 'm5.xlarge'),
 ('aws:us-east-1', 'azure:uaenorth'): (1, 2, 'm5.large'),
 ('aws:us-east-1', 'gcp:us-east1'): (4, 4, 'm5.xlarge'),
 ('azure:uaenorth', 'aws:us-east-1'): (1, 2, 'Standard_D2_v5'),
 ('azure:uaenorth', 'azure:uaenorth'): (1, 2, 'Standard_D2_v5'),
 ('azure:uaenorth', 'gcp:us-east1'): (1, 2, 'Standard_D2_v5'),
 ('gcp:us-east1', 'aws:us-east-1'): (4, 4, 'n2-standard-4'),
 ('gcp:us-east1', 'azure:uaenorth'): (1, 2, 'n2-standard-2'),
 ('gcp:us-east1', 'gcp:us-east1'): (2, 8, 'n2-standard-8')}

In [18]:
%load_ext autoreload
%autoreload 2

In [27]:
%pwd

'/Users/erenerdogan/skyplane/skyplane'

In [29]:
import csv

data = [
    # AWS
    ["m5.8xlarge", "aws", 32],
    ["m5.12xlarge", "aws", 48],
    ["m5.24xlarge", "aws", 96],
    ["m5.xlarge", "aws", 4],
    ["m5.4xlarge", "aws", 16],
    ["m5.large", "aws", 2],
    ["m5.2xlarge", "aws", 8],
    ["m5.16xlarge", "aws", 64],
    
    # GCP
    ["n2-standard-128", "gcp", 128],
    ["n2-standard-96", "gcp", 96],
    ["n2-standard-80", "gcp", 80],
    ["n2-standard-64", "gcp", 64],
    ["n2-standard-48", "gcp", 48],
    ["n2-standard-32", "gcp", 32],
    ["n2-standard-16", "gcp", 16],
    ["n2-standard-8", "gcp", 8],
    ["n2-standard-4", "gcp", 4],
    ["n2-standard-2", "gcp", 2],

    # Azure
    ["Standard_D96_v5", "azure", 96],
    ["Standard_D80_v5", "azure", 80],
    ["Standard_D64_v5", "azure", 64],
    ["Standard_D48_v5", "azure", 48],
    ["Standard_D32_v5", "azure", 32],
    ["Standard_D16_v5", "azure", 16],
    ["Standard_D8_v5", "azure", 8],
    ["Standard_D4_v5", "azure", 4],
    ["Standard_D2_v5", "azure", 2]
]

filename = "./data/vcpu_info.csv"

with open(filename, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['instance_name', 'cloud_provider', 'vcpu_cost'])  # Writing the header row
    writer.writerows(data)  # Writing the data rows

print(f"CSV file '{filename}' created successfully.")

CSV file './data/vcpu_info.csv' created successfully.


In [36]:
import csv
from importlib.resources import path

file_path = path("skyplane.data", "vcpu_info.csv")
data_list = ()

print(file_path)

with path("skyplane.data", "vcpu_info.csv") as file_path:
    with open(file_path, "r") as csvfile:
        reader = csv.reader(csvfile)
        next(reader)  # Skip the header row

        for row in reader:
            instance_name, cloud_provider, vcpu_cost = row
            data_list.append((instance_name, cloud_provider, int(vcpu_cost)))

('m5.8xlarge', 'aws', 32)
('m5.12xlarge', 'aws', 48)
('m5.24xlarge', 'aws', 96)
('m5.xlarge', 'aws', 4)
('m5.4xlarge', 'aws', 16)
('m5.large', 'aws', 2)
('m5.2xlarge', 'aws', 8)
('m5.16xlarge', 'aws', 64)
('n2-standard-128', 'gcp', 128)
('n2-standard-96', 'gcp', 96)
('n2-standard-80', 'gcp', 80)
('n2-standard-64', 'gcp', 64)
('n2-standard-48', 'gcp', 48)
('n2-standard-32', 'gcp', 32)
('n2-standard-16', 'gcp', 16)
('n2-standard-8', 'gcp', 8)
('n2-standard-4', 'gcp', 4)
('n2-standard-2', 'gcp', 2)
('Standard_D96_v5', 'azure', 96)
('Standard_D80_v5', 'azure', 80)
('Standard_D64_v5', 'azure', 64)
('Standard_D48_v5', 'azure', 48)
('Standard_D32_v5', 'azure', 32)
('Standard_D16_v5', 'azure', 16)
('Standard_D8_v5', 'azure', 8)
('Standard_D4_v5', 'azure', 4)
('Standard_D2_v5', 'azure', 2)
